# A/B Testing for ShoeFly.com

The goal of this project is to analyze the data, using Pandas aggregate measures, for an online shoe store ShoeFly.com that is performing an A/B testing. The shop has two different versions of an ad, which they have placed in emails, as well as in banner ads on Facebook, Twitter, and Google. They want to know how the two ads are performing on each of the different platforms on each day of the week. 

## Analyzing Ad Sources

In [11]:
# Import Pandas library
import pandas as pd
# Read the the file with the dataset
ad_clicks = pd.read_csv('ad_clicks.csv')
# Display first 10 rows
ad_clicks.head(10)


,user_id,utm_source,day,ad_click_timestamp,experimental_group
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,7:18,A
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,NaN,B
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,NaN,A
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,NaN,B
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,NaN,B
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,NaN,A
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,NaN,A
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,NaN,A
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,18:33,A
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,15:21,B


A manager wants to know how much views getting each ad platform and which is getting the most.

In [12]:
utm_source_views = ad_clicks.groupby('utm_source').user_id.count().reset_index()
utm_source_views.rename(columns ={
  'user_id':'number_of_views'
}, inplace = True)
utm_source_views

,utm_source,number_of_views
0,email,255
1,facebook,504
2,google,680
3,twitter,215


The manager wants to know who amongst users clicked on the add and who did not.

We are going to create a new column called **is_click**, which is True if **ad_click_timestamp** is not NaN and False otherwise. If the column **ad_click_timestamp** is not NaN, then someone actually clicked on the ad that was displayed.


In [13]:
ad_clicks['is_click'] = ad_clicks.ad_click_timestamp.notna()
# Change columns order
another_order_of_ad_clicks = ad_clicks[['user_id', 'utm_source', 'day', 'experimental_group', 'is_click', 'ad_click_timestamp', ]]
# Display first 10 rows
another_order_of_ad_clicks.head(10)

,user_id,utm_source,day,experimental_group,is_click,ad_click_timestamp
0,008b7c6c-7272-471e-b90e-930d548bd8d7,google,6 - Saturday,A,True,7:18
1,009abb94-5e14-4b6c-bb1c-4f4df7aa7557,facebook,7 - Sunday,B,False,NaN
2,00f5d532-ed58-4570-b6d2-768df5f41aed,twitter,2 - Tuesday,A,False,NaN
3,011adc64-0f44-4fd9-a0bb-f1506d2ad439,google,2 - Tuesday,B,False,NaN
4,012137e6-7ae7-4649-af68-205b4702169c,facebook,7 - Sunday,B,False,NaN
5,013b0072-7b72-40e7-b698-98b4d0c9967f,facebook,1 - Monday,A,False,NaN
6,0153d85b-7660-4c39-92eb-1e1acd023280,google,4 - Thursday,A,False,NaN
7,01555297-d6e6-49ae-aeba-1b196fdbb09f,google,3 - Wednesday,A,False,NaN
8,018cea61-19ea-4119-895b-1a4309ccb148,email,1 - Monday,A,True,18:33
9,01a210c3-fde0-4e6f-8efd-4f0e38730ae6,email,2 - Tuesday,B,True,15:21


The manager wants to the percent of people who clicked on ads from each **utm_source**. We need to group results by **utm_source** and **is_click**, and count the results in each of those groups to create pivot table with resuls.

In [14]:
# Group by source and clicks
clicks_by_source = another_order_of_ad_clicks.groupby(['utm_source', 'is_click'])['user_id'].count().reset_index()
# Create pivot table
clicks_pivot = clicks_by_source.pivot(
  columns = 'is_click',
  index = 'utm_source',
  values = 'user_id'
)
# Add a new column with percentages of those who clicked
ratio_to_1 = lambda row: row[True] / (row[False] + row[True])
clicks_pivot['percent_of_those_who_clicked'] = clicks_pivot.apply(ratio_to_1, axis = 1)
# Format the new column's values to display them as %
clicks_pivot.style.format(formatter={'percent_of_those_who_clicked': '{:.2%}'})

is_click,0,1,percent_of_those_who_clicked
utm_source,,,
email,175,80,31.37%
facebook,324,180,35.71%
google,441,239,35.15%
twitter,149,66,30.70%


## Analyzing an A/B Test

The column experimental_group tells us whether the user was shown Ad A or Ad B. Manager wants to know wheather the the number of people in 2 groups was the same. Results of the calculation says the number of people was the same.

In [15]:
# Count thr number of people in each group
ad_clicks.groupby('experimental_group').user_id.count()

experimental_group
A    827
B    827
Name: user_id, dtype: int64

Manager asked to check if a greater percentage of users clicked on Ad A or Ad B. According to the results below - **A** got more clicks by approximately `7%`.

In [16]:
# Group values by exp. group and click or no click
clicks_by_exp_group = ad_clicks.groupby(['experimental_group', 'is_click']).user_id.count().reset_index()
# Format as pivot
clicks_by_exp_group_pivot = clicks_by_exp_group.pivot(
  columns = 'is_click',
  index = 'experimental_group',
  values = 'user_id'
)
# Add % column
clicks_by_exp_group_pivot['percent_of_those_who_clicked'] = clicks_by_exp_group_pivot.apply(ratio_to_1, axis = 1)
# Rename columns
clicks_by_exp_group_pivot.rename(columns= {
  0: 'False',
  1: 'True'
  }, inplace = True)
# Format the new column's values to display them as %
clicks_by_exp_group_pivot.style.format(formatter={'percent_of_those_who_clicked': '{:.2%}'})

is_click,False,True,percent_of_those_who_clicked
experimental_group,,,
A,517,310,37.48%
B,572,255,30.83%


The Product Manager for the A/B test thinks that the clicks might have changed by day of the week. Let's check this out.

In [17]:
# Create and group by days and clicks DataFrame for A
a_clicks = ad_clicks[ad_clicks.experimental_group == 'A'].groupby(['day', 'is_click']).user_id.count().reset_index()
# Format A DataFrame as pivot
a_clicks_pivot = a_clicks.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
)
# Add % column to A
a_clicks_pivot['percent_of_those_who_clicked'] = a_clicks_pivot.apply(ratio_to_1, axis = 1)

# Create and group by days and clicks DataFrame for B
b_clicks = ad_clicks[ad_clicks.experimental_group == 'B'].groupby(['day', 'is_click']).user_id.count().reset_index()
# Format B DataFrame as pivot
b_clicks_pivot = b_clicks.pivot(
  columns = 'is_click',
  index = 'day',
  values = 'user_id'
)
# Add % column to B
b_clicks_pivot['percent_of_those_who_clicked'] = b_clicks_pivot.apply(ratio_to_1, axis = 1)

# Compare DataFrames A and B
c = a_clicks_pivot.compare(b_clicks_pivot)
# Rename columns in the comparison DataFrame
c.columns = c.columns.set_levels([['False', 'True', 'percent_of_those_who_clicked'], ['A', 'B']])
# Function for rows' styling
def make_pretty(row):
  a_prop = ''
  b_prop = 'background: {background}; color:black;'.format(background='#a7dd87' if int(row.name[0]) % 2 else '#c0ff9b')
  if row['percent_of_those_who_clicked']['A'] > row['percent_of_those_who_clicked']['B']:
    a_prop, b_prop = b_prop, a_prop
  return [None, None, None, None, a_prop, b_prop]
# Apply style and format the 'percent_of_those_who_clicked' values to display them as %
c.style.apply(make_pretty, axis=1).format(formatter={('percent_of_those_who_clicked', 'A'): '{:.2%}', ('percent_of_those_who_clicked', 'B'): '{:.2%}'})


### Conclusion

Ad A performed better on all days of the week except Tuesday, therefore it is recommended to use it as a final version. 